In [ ]:
import pandas as pd
from pathlib import Path
from storage.cherrytree import CherryTree
from utility.config import load_config
import sys
import redis
import dateparser

cf = load_config()
namespace = cf.get('namespace', 'document')
base_node = cf.get('base-node', None)
r = redis.Redis(decode_responses=True)


In [ ]:
def load_status_data():
    ct = CherryTree(cf['index-file']) 
    di = f'{namespace}:filepath.document:index'
    data = []
    for node in [n for n in ct.nodes(base_node) if n.filepath]:
        dk = r.hget(di, node.filepath)
        if dk:
            data.append(r.hgetall(dk))
        else:
            print(dk, 'not found')
    
    df = pd.DataFrame(data).fillna('No Data')
    df['timestamp'] = df.date.apply(lambda x: dateparser.parse(x))
    return df

In [ ]:
df =load_status_data()

In [4]:
df[['title', 'date']]

,title,date
0,Sabam Assigns Muharto Interview,27 September 1988
1,Cameron Interviews Muharto,28 September 1988
2,Surprise Call,7 June 1947
3,Flight to Tasik,28 September 1988
4,Third CALI Run,31 March 1947
...,...,...
123,Karangendah Raid,December 1988
124,Syd Absent,December 1988
125,Sabam Fires Cameron,December 1988
126,Victor Learns Truth,December 1988
